In [1]:
#pip install openpyxl

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 10 10:10:58 2020

@author: hyeminum / gpinedaortiz
"""
import glob
import logging
import os
import numpy as np
import pandas as pd
import skimage as ski
import itertools  
from itertools import zip_longest 

from skimage import data, util
from skimage.measure import label, regionprops

FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)


# Setting file naming and strings

In [152]:
label_val = 2
# check for every database (Brats convention)
if label_val == 4:
    habitat = 'Enhancing'
elif label_val == 2:
    habitat = 'Edema'
elif label_val == 1:
    habitat = 'Necrosis'
elif label_val == 0:
    habitat = 'Habitat'
habitat

'Enhancing'

In [153]:
mri_data='xCures'
subdir='Feature_extraction'
time_point='Baseline'
feature='shape_global'
feature_str= 'All_label_brats_SRI24_label_' + str(label_val) +'.txt'

## untar (tar.gz) deformation features (containing masks in MNISpace)

In [154]:
def getting_feature_list(mri_data,subdir,time_point,feature_str):

    root_path = '/app/Data/_Brain/Radiology/_Adult/_Glioma/'
    outpath_base= os.path.join(root_path + mri_data + '/Feature_extraction/Feature_matrix',time_point)
    print('main outpath ....... ',outpath_base)
    main_path=os.path.join(root_path , mri_data, subdir,'Radiomics_features','*',time_point, feature, '*' + feature_str)
    print('main_path tumor labels.......',main_path)
    
    # masks with all the tumor labels
    labels = glob.glob(main_path)
    labels=sorted(labels)
    print('\n number of features files: {}'.format(len(labels)))
 
    return outpath_base, main_path, labels

In [155]:
# extract mask form deformation features in MNI space
def unzip_deform_features(path):
    path = '/app/Data/_Brain/Radiology/_Adult/_Glioma/xCures/Preprocessed/'
    #path = "/app/IDIA/MRI/GBM/xCures/Radiomics/"

    %cd $path

    files = glob.glob(path +'coregist*tar.gz')
    files = sorted(files)
    for items in range(0,len(files)):
        print(os.path.basename(files[items]))
        file_name=os.path.basename(files[items])
        ! tar -xzvf $file_name -C Pre-op-Post-op --strip-components 1
        #! tar -tf $file_name 
    
    return files

In [156]:
outpath_base, main_path, labels=getting_feature_list(mri_data,subdir,time_point,feature_str)

main outpath .......  /app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Feature_matrix/Baseline
main_path tumor labels....... /app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/*/Baseline/shape_global/*All_label_brats_SRI24_label_4.txt

 number of features files: 33


In [157]:
outname= os.path.join(outpath_base, mri_data +'_' + habitat + '_global_shape_pp2.xlsx')
outname

'/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Feature_matrix/Baseline/IVYGAP_Enhancing_global_shape_pp2.xlsx'

## Global Shape features (separated masks for tumor compartments)

In [158]:
labels

['/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W01/Baseline/shape_global/W01_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W02/Baseline/shape_global/W02_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W04/Baseline/shape_global/W04_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W05/Baseline/shape_global/W05_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W07/Baseline/shape_global/W07_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Radiomics_features/W08/Baseline/shape_global/W08_01_All_label_brats_SRI24_label_4.txt',
 '/app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/F

In [159]:
props_names=list(['Volume','Integrated Intensity','Centroid_normalized1','Centroid_normalized2','Centroid_normalized3',
	'Weighted Centroid1','Weighted Centroid2','Weighted Centroid3',
    'Axes_Length1','Axes_Length2','Axes_Length3',
	'MajorAxisLength','MinorAxisLength','Eccentricity',
	'Elongation','Orientation_radians','xmin', 'xmax', 'ymin', 'ymax', 'zmin', 'zmax',
	'PrincipalMoments_Eigen1','PrincipalMoments_Eigen2','PrincipalMoments_Eigen3',
    'Perimeter_surface_3D','Roundness','EquivalentSphericalRadius',
	'EquivalentSphericalPerimeter_surface','EquivalentEllipsoidDiameter1',
    'EquivalentEllipsoidDiameter2','EquivalentEllipsoidDiameter3',
	'Flatness','Elongation_Shape_Factor','NumberOfPixelsOnBorder','PerimeterOnBorder',
    'PerimeterOnBorderRatio','Tumor Compactness'])
print('len props names ..... ',len(props_names))

len props names .....  38


In [160]:
feature_mat1=[]
feature_mat2=[]
feature_mat3=[]
ids1=[]
idx_df1=[]
rows=[]
labels_id=[]
it=-1
for i in labels:
# for i in range(it,10):
    it=it+1
    rows=[]
    with open(i, 'r') as f:
        data = f.read()        
    rows = data.split('\n')
    if len(rows)==25:
        labels_id=np.array([1])

    print('computing on .............................. ', os.path.basename(i))
    case_id=os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(i))))
    print('case id .............................. ', case_id)
    for id in labels_id:
        id_arr=rows[id:id+23]
        new_arr=[]
        it2=-1
        for s in id_arr:
            it2=it2+1
            vector_str = s.replace("[", "").replace("]", "")
            vector_list=(vector_str.split(","))
            vector_float = np.array(vector_list, dtype=float)
            if it2==0:
                new_arr=vector_float
            elif it2>0:
                new_arr=np.hstack((new_arr,vector_float))
            new_arr2=np.array(new_arr,dtype=object)
        if id==1:
            feature_mat1.append(new_arr2)
            ids1.append(case_id)
            idx_df1.append(case_id)

computing on ..............................  W01_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W01
computing on ..............................  W02_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W02
computing on ..............................  W04_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W04
computing on ..............................  W05_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W05
computing on ..............................  W07_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W07
computing on ..............................  W08_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W08
computing on ..............................  W09_01_All_label_brats_SRI24_label_4.txt
case id ..............................  W09
computing on ..............................  W10_01_All_label_brats_SRI24_label_4.txt
case

In [161]:
feature_mat1[0].shape

(38,)

In [162]:
df_shape = pd.DataFrame(np.array(feature_mat1), index = ids1, columns = props_names)
df_idx = pd.DataFrame(np.squeeze(idx_df1), index = ids1)

with pd.ExcelWriter(outname) as writer:
                df_shape.to_excel(writer, sheet_name="FeatureMatrix",index_label='ID')
                df_idx.to_excel(writer, sheet_name="df_idx",index_label='list_id')
print('saved in ...... ',  outname)

saved in ......  /app/Data/_Brain/Radiology/_Adult/_Glioma/IVYGAP/Feature_extraction/Feature_matrix/Baseline/IVYGAP_Enhancing_global_shape_pp2.xlsx
